In [2]:
#!pip install pyunicore --upgrade
! pip install pyunicore==0.2.1

  Running setup.py bdist_wheel for pyunicore ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/e4/7f/33/dc8215617bf94ef7ccf1c6e658c5eb528efb73a26634fae9b2
Successfully built pyunicore
  Found existing installation: pyunicore 0.5.5
    Uninstalling pyunicore-0.5.5:
      Successfully uninstalled pyunicore-0.5.5
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [21]:
import pyunicore.client as unicore_client
tr = unicore_client.Transport(oauth.get_token())
r = unicore_client.Registry(tr, unicore_client._HBP_REGISTRY_URL)
# check access level
print r.site_urls
client = r.site("HBP_JULIA")
access_info = client.access_info()
role = access_info['role']['selected']
print("Access level: '%s'" % (role))
if role=="user":
    print("  Unix ID(s) : %s" % access_info['xlogin']['availableUIDs'])

{u'JURON': u'https://zam2125.zam.kfa-juelich.de:9112/JURON/rest/core', u'BSC-MareNostrum': u'https://unicore-hbp.bsc.es:8080/BSC-MareNostrum/rest/core', u'DAINT-CSCS': u'https://brissago.cscs.ch:8080/DAINT-CSCS/rest/core', u'RWTH': u'https://unicore.hpc.itc.rwth-aachen.de:8080/RWTH/rest/core', u'FZJ_JURECA': u'https://zam2125.zam.kfa-juelich.de:9112/FZJ_JURECA/rest/core'}


KeyError: 'HBP_JULIA'

In [17]:
download_url = "https://services.humanbrainproject.eu/storage/v1/api/file/a2850a99-c939-445d-b2cb-f85f318b7f70/content"
job = {}

job['Executable'] = "./reachgrasp-spikewave/src/calc_waveproperties.py"
job['Arguments'] = ["0"]
job['Parameters'] = {}
job['Execution environment'] = {
    "Name": "PYTHON",
    "User precommand": "mkdir datasets; mv i140703* datasets/ ; tar xzf files.tar.gz ;"+
           "virtualenv venv ; source venv/bin/activate; "+ 
           "pip install pip --upgrade; "+
           "module load libpng/1.5.28 ;"+
           "pip install -r reachgrasp-spikewave/src/requirements.txt",
    "RunUserPrecommandOnLoginNode": "true",
}
job['Resources'] = {
    "Nodes": "1",
#    "ArraySize": "8",
}

job["haveClientStageIn"] = "false"

# add stage-ins
job['Imports'] = [
    {"From": "https://web.gin.g-node.org/INT/multielectrode_grasp/raw/master/datasets/i140703-001.ns2",
     "To": "i140703-001.ns2",
    },
    {"From": "https://web.gin.g-node.org/INT/multielectrode_grasp/raw/master/datasets/i140703-001.nev",
     "To": "i140703-001.nev",
    },
    {
     "From": download_url,
     "To": "files.tar.gz",
     "Credentials": {"BearerToken":""},
    }
]

job

{'Arguments': ['0'],
 'Executable': './reachgrasp-spikewave/src/calc_waveproperties.py',
 'Execution environment': {'Name': 'PYTHON',
  'RunUserPrecommandOnLoginNode': 'true',
  'User precommand': 'mkdir datasets; mv i140703* datasets/ ; tar xzf files.tar.gz ;virtualenv venv ; source venv/bin/activate; pip install pip --upgrade; module load libpng/1.5.28 ;pip install -r reachgrasp-spikewave/src/requirements.txt'},
 'Imports': [{'From': 'https://web.gin.g-node.org/INT/multielectrode_grasp/raw/master/datasets/i140703-001.ns2',
   'To': 'i140703-001.ns2'},
  {'From': 'https://web.gin.g-node.org/INT/multielectrode_grasp/raw/master/datasets/i140703-001.nev',
   'To': 'i140703-001.nev'},
  {'Credentials': {'BearerToken': ''},
   'From': 'https://services.humanbrainproject.eu/storage/v1/api/file/a2850a99-c939-445d-b2cb-f85f318b7f70/content',
   'To': 'files.tar.gz'}],
 'Parameters': {},
 'Resources': {'Nodes': '1'},
 'haveClientStageIn': 'false'}

In [13]:
# submit a job
job = client.new_job(job)
job

Job: 5b6905cb-6250-4741-880d-5ad218e0bf85: 5b6905cb-6250-4741-880d-5ad218e0bf85, submitted: 2018-02-22T11:16:40+0100 running: True

In [ ]:
print(job.properties)

In [ ]:
print(job.working_dir.properties)

In [ ]:
%%writefile -a test.py

import sys
print (sys.version)

In [ ]:
# upload it
job.working_dir.upload("./test.py")

In [ ]:
job.working_dir.listdir()

In [ ]:
# start after uploading all required files
job.start()

In [36]:
d_client = get_bbp_client().document
uid = d_client.upload_file("./test.py", "/5185/test.py","text/plain")
print(uid)


d2fb8b01-9db0-4453-8b50-4f20f76e5f42


In [35]:
d_client.remove("/5185/test.py")

In [40]:
d_client.get_standard_attr("/5185/test.py")

{'_contentType': 'text/plain',
 '_contentUri': '',
 '_createdBy': '252298',
 '_createdOn': '2018-02-20T16:20:49.066312Z',
 '_description': '',
 '_entityType': 'file',
 '_name': 'test.py',
 '_parent': '854bf137-f286-4f1c-b07c-b7064ca1faf5',
 '_uuid': 'd2fb8b01-9db0-4453-8b50-4f20f76e5f42'}

In [25]:
storage_api = "https://services.humanbrainproject.eu/storage/v1/api/file/"
download_url = storage_api + uid + "/content"

job = {"haveClientStageIn": "false"}
job['Imports'] = [
    
    {"From": download_url, 
     "To" : "test.py",
     
     # server already has the OAuth token :)
     "Credentials" : {"BearerToken": ""},
    },
]

job

{'Imports': [{'Credentials': {'BearerToken': ''},
   'From': 'https://services.humanbrainproject.eu/storage/v1/api/file/e41d01f3-1c02-4b63-9242-52a3b19e428e/content',
   'To': 'test.py'}],
 'haveClientStageIn': 'false'}

In [26]:
# run this
job = client.new_job(job)
job.properties

{u'_links': {u'action:abort': {u'description': u'Abort',
   u'href': u'https://hbp-unic.fz-juelich.de:7112/HBP_JULIA/rest/core/jobs/6326e783-e39d-44f7-b0c7-6a2e1f60dc81/actions/abort'},
  u'action:restart': {u'description': u'Restart',
   u'href': u'https://hbp-unic.fz-juelich.de:7112/HBP_JULIA/rest/core/jobs/6326e783-e39d-44f7-b0c7-6a2e1f60dc81/actions/restart'},
  u'action:start': {u'description': u'Start',
   u'href': u'https://hbp-unic.fz-juelich.de:7112/HBP_JULIA/rest/core/jobs/6326e783-e39d-44f7-b0c7-6a2e1f60dc81/actions/start'},
  u'parentTSS': {u'description': u'Parent TSS',
   u'href': u'https://hbp-unic.fz-juelich.de:7112/HBP_JULIA/rest/core/sites/26bcbfea-69af-43e5-96f5-808d9634057c'},
  u'self': {u'href': u'https://hbp-unic.fz-juelich.de:7112/HBP_JULIA/rest/core/jobs/6326e783-e39d-44f7-b0c7-6a2e1f60dc81'},
  u'workingDirectory': {u'description': u'Working directory',
   u'href': u'https://hbp-unic.fz-juelich.de:7112/HBP_JULIA/rest/core/storages/6326e783-e39d-44f7-b0c7-6a2e1

In [27]:
job.working_dir.listdir()

{u'test.py': PathFile: test.py}

In [28]:
x = job.working_dir.stat("test.py").raw().read()
x

'\nimport sys\nprint (sys.version)\nimport sys\nprint (sys.version)\nimport sys\nprint (sys.version)'

In [1]:
import snakemake

ImportError: No module named snakemake

In [1]:
!pip install snakemake

  Using cached https://files.pythonhosted.org/packages/f4/a1/8b5697151dab2d96ac1e39c9843c97027d210f6a241515436b0eeb0ab5e1/snakemake-5.1.4.tar.gz
    100% |████████████████████████████████| 194kB 1.9MB/s ta 0:00:011
    100% |████████████████████████████████| 552kB 920kB/s ta 0:00:011
  Running setup.py bdist_wheel for snakemake ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/a7/73/42/32544d7ec11f99d65a3a0ec0df89b5b5c94da1c54cb6c59705
  Running setup.py bdist_wheel for wrapt ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/48/5d/04/22361a593e70d23b1f7746d932802efe1f0e523376a74f321e
  Running setup.py bdist_wheel for configargparse ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/76/11/eb/12113becd46c0e8a70b8a63f9405e46a1f61d4aaa8532d676b
  Running setup.py bdist_wheel for datrie ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/2b/8b/31/fe6fb3dd47e34a4b8cb7505c501676f962f733f0ed9885ede7
Successfully built snakemake wrapt configargp

In [3]:
import snakemake

In [7]:
%%writefile a.smk
rule all:
    output: 
        "atest"
    shell:
        """
        touch atest
        """

Overwriting a.smk


In [11]:
snakemake.snakemake("a.smk")

Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	all
	1

rule all:
    output: atest
    jobid: 0

Finished job 0.
1 of 1 steps (100%) done
Complete log: /home/jovyan/.snakemake/log/2018-06-11T102941.698253.snakemake.log


True

In [12]:
!ls

a.smk  atest  work


In [13]:
!rm atest

In [ ]:
#TODO check why pyxus doesn't resolve its dependencies 

! pip install pyld
! pip install rdflib
! pip install pystache
! pip install rdflib-jsonld
! pip install curlify
! pip install --index-url https://testpypi.python.org/pypi pyxus
    

In [ ]:
# Get your access token to authorize against the nexus platform and instantiate the nexus client

from openid_http_client.auth_client.access_token_client import AccessTokenClient
from pyxus.client import NexusClient

import logging

logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [1]:
from jupyter_collab_storage import oauth_token_handler as oauth
access_token_client = AccessTokenClient(oauth.get_token())

#Please note - the endpoint definition is optional - if it is not given, Pyxus tries to fetch the values from the system environment variables (which is mainly for server-side scripts)
nexus = NexusClient(scheme="https", host="nexus-int.humanbrainproject.org", prefix="v0", alternative_namespace="https://nexus-int.humanbrainproject.org", auth_client=access_token_client)

NameError: name 'AccessTokenClient' is not defined

In [3]:
import json

organization_list = nexus.organizations.list(resolved=True, )
print "{} items found".format(organization_list.total)

for organization in organization_list.results:
    print json.dumps(organization.data, indent=4)
    print "\n********************************\n"

SyntaxError: invalid syntax (<ipython-input-3-d9a0af49b471>, line 4)

In [11]:
# List your data instances

# ATTENTION: The number of displayed instances depends on your access rights!

import json

instances_list = nexus.instances.list(resolved=True, full_text_query="neural*")
print "{} items found".format(instances_list.total)

for instance in instances_list.results:
    print json.dumps(instance.data, indent=4)
    print "\n********************************\n"

57 items found
{
    "nxv:deprecated": false, 
    "nxv:rev": 6, 
    "links": {
        "@context": "https://nexus-int.humanbrainproject.org/v0/contexts/nexus/core/links/v0.2.0", 
        "outgoing": "https://nexus-int.humanbrainproject.org/v0/data/minds/core/placomponent/v0.0.4/03bb0eb9-4f26-4dfa-a073-9261bcb18f07/outgoing", 
        "incoming": "https://nexus-int.humanbrainproject.org/v0/data/minds/core/placomponent/v0.0.4/03bb0eb9-4f26-4dfa-a073-9261bcb18f07/incoming", 
        "self": "https://nexus-int.humanbrainproject.org/v0/data/minds/core/placomponent/v0.0.4/03bb0eb9-4f26-4dfa-a073-9261bcb18f07", 
        "schema": "https://nexus-int.humanbrainproject.org/v0/schemas/minds/core/placomponent/v0.0.4"
    }, 
    "http://hbp.eu/internal#hashcode": "70c3097c8dfad25bfcc29caeb7574cbd", 
    "http://www.w3.org/ns/prov#qualifiedAssociation": {
        "http://www.w3.org/ns/prov#agent": {
            "@id": "https://nexus-int.humanbrainproject.org/v0/data/minds/core/softwareagent/v0.0.

In [12]:


# List your data instances based on its schema filtered by a fulltext query

# ATTENTION: The number of displayed instances depends on your access rights!

import json

instances_list = nexus.instances.list(subpath="/minds/core/dataset/v0.0.4", full_text_query="Massively*", resolved=True)
print "{} items found".format(instances_list.total)

for instance in instances_list.results:
    print json.dumps(instance.data, indent=4)
    print "\n********************************\n"



1 items found
{
    "links": {
        "@context": "https://nexus-int.humanbrainproject.org/v0/contexts/nexus/core/links/v0.2.0", 
        "outgoing": "https://nexus-int.humanbrainproject.org/v0/data/minds/core/dataset/v0.0.4/a305b192-8c08-4fa4-8a2a-3524be2e488c/outgoing", 
        "incoming": "https://nexus-int.humanbrainproject.org/v0/data/minds/core/dataset/v0.0.4/a305b192-8c08-4fa4-8a2a-3524be2e488c/incoming", 
        "self": "https://nexus-int.humanbrainproject.org/v0/data/minds/core/dataset/v0.0.4/a305b192-8c08-4fa4-8a2a-3524be2e488c", 
        "schema": "https://nexus-int.humanbrainproject.org/v0/schemas/minds/core/dataset/v0.0.4"
    }, 
    "http://hbp.eu/minds#parcellationAtlas": {
        "@id": "https://nexus-int.humanbrainproject.org/v0/data/minds/core/parcellationatlas/v0.0.4/aa6b56a2-8fa4-4731-81fa-ae9da19ecca0"
    }, 
    "http://hbp.eu/minds#activity": {
        "@id": "https://nexus-int.humanbrainproject.org/v0/data/minds/core/activity/v0.0.4/79cc92a3-c115-4feb-96d3

In [18]:
print instance.data[u'http://hbp.eu/minds#datalink']
print instance.data[u'http://hbp.eu/minds#datasetDOI']

https://web.gin.g-node.org/INT/multielectrode_grasp
10.12751/g-node.f83565
